In [3]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import json
import urllib.request
from torchvision import models, transforms

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model_path = "./vgg19.tv_in1k/pytorch_model.bin"
state_dict = torch.load(model_path, map_location=torch.device('cpu'))#timm.create_model('vgg19', pretrained=True)

model = timm.create_model('vgg19', pretrained=False)
model.load_state_dict(state_dict)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)
print(top5_probabilities)


# Load the ImageNet class index
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
class_index = json.loads(urllib.request.urlopen(url).read().decode())

# Get the top 5 class indices and probabilities
top5_class_indices = top5_class_indices.squeeze().tolist()
top5_probabilities = top5_probabilities.squeeze().tolist()

# Print the top 5 class labels and probabilities
for i in range(5):
    class_index_str = class_index[str(top5_class_indices[i])]
    class_label = class_index_str[1]
    print(f"{class_label}: {top5_probabilities[i]:.2f}%")

tensor([[27.9808, 25.2350, 11.8262,  3.8008,  3.0933]],
       grad_fn=<TopkBackward0>)
ice_cream: 27.98%
trifle: 25.24%
chocolate_sauce: 11.83%
eggnog: 3.80%
ladle: 3.09%
